# Voice enabled room control system
   

---

## Aim

* This application notebook shows working of a voice based room control system for controlling the ambience of a room.   
  It uses an online service to recognize the given speech based commands to control various sensors and responds back to the user with an appropriate reply.


## References 
* [Grove temperature](https://www.seeedstudio.com/Grove-Temperature-Sensor.html) 
* [Grove Servo](https://wiki.seeedstudio.com/Grove-Servo.html)  
* [Grove Relay](https://www.seeedstudio.com/Grove-Relay.html)  
* [Grove I2C OLED](https://wiki.seeedstudio.com/Grove-OLED_Display_0.96inch/) 
* [Grove RGB LED Stick](https://www.seeedstudio.com/Grove-RGB-LED-Stick-10-WS2813-Mini.html) 
* [Grove Base Shield V2.0](https://www.seeedstudio.com/Base-Shield-V2.html)   
* [Speech Recognition](https://pypi.org/project/SpeechRecognition/)
* [Text to Speech](https://gtts.readthedocs.io/en/latest/)

## Last revised
* 07 April 2021
    + Initial version
---

<div class="alert alert-box alert-warning">
   <h4 class="alert-heading">Notes before running notebook </h4>
    <ul>
        <li> Please run the notebook while connected to regular wall power. As many modules are used in the application the USB power may not be sufficient</li>
        <li> Connect to Internet (wired or wireless) before running the code cells as we use free online services</li>
        <li> You will need Earphones+Mic to give voice commands to the application</li>
    </ul> 
</div>

In [1]:
import os
import re
from time import sleep

from IPython.display import display, HTML, Audio, clear_output
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter, PmodGroveAdapter

<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Library Dependencies </h4>
Before proceeding the following libraries must be installed
    <ul>
        <li> speech_recognition </li>
        <li> flac</li>
        <li> portaudio19-dev</li>
        <li> pyaudio</li>
        <li> gtts </li>
    </ul>
</div>

In [2]:
try:
    import speech_recognition as sr
except ImportError:
    print("This script requires the the following module installs: \n"
          "1. FLAC : sudo apt-get install flac \n"
          "2. portaudio19-dev : sudo apt-get install portaudio19-dev \n"
          "3. pyaudio : sudo pip3 install pyaudio\n")
    print("Then install speech_recognition module with: \n"
         "sudo pip3 install SpeechRecognition")
try:
    from gtts import gTTS
except ImportError:
    print("This script requires the gtts module \n Install with: sudo "
         "pip3 install gtts")

---

## Constructing application with Grove Base Shield V2.0 (Arduino)

### Library compilation

In [3]:
base = BaseOverlay("base.bit")
r = sr.Recognizer()

pAudio = base.audio
pAudio.set_volume(20)
pAudio.select_microphone()
pAudio.bypass(seconds=5)

adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, D6='grove_relay',
                                   I2C='grove_oled', A1='grove_temperature',
                                   D7='grove_led_stick', D5='grove_servo')

### Create all the Grove devices

In [4]:
relay = adapter.D6
oled = adapter.I2C
temp_sensor = adapter.A1
led_stick = adapter.D7
servo = adapter.D5

<div class="alert alert-box alert-danger">
    <h4 class="alert-heading">Stop before proceeding further! </h4>
     Please make sure you had connected all the sensors before switching the board ON. If the sensors are connected while the board is ON it can cause the system to shutdown. For safe use, you can note down the connections shown below, switch the board OFF make the physical connections and the switch the board back ON.
    
   <h4 class="alert-heading">Make Physical Connections </h4>
    <ul>
        <li>Insert the Grove Base Shield into the Arduino connector on the board.</li>
        <li>Connect the grove_temperature module to A1 connector of the Grove Base Shield.</li>
        <li>Connect the grove_servo module to D5 connector of the Grove Base Shield.</li>
        <li>Connect the grove_led_stick module to D7 connector of the Grove Base Shield.</li>
        <li>Connect the grove_oled module to one of the I2C connector of the Grove Base Shield.</li>
        <li> Connect the grove_relay module to D6 connector of the Grove Base Shield.</li></ul>
</div>

![](data/room.png)

### Construct the voice enabled room control system

In [7]:
global sensor_state

oled.set_default_config()
oled.set_normal_display()
oled.set_next_row_wrap_mode()
led_stick.clear()

led_colors = [0xFF0000, 0xFF2300, 0xFF5700, 0xFF8C00, 0xFFC100, 0xFFF600,
              0xD4FF00, 0x9FFF00, 0x6AFF00, 0x00FF00]

sensor_state = {'relay': 'off', 'servo': 'off'}


def t2s(text):
    try:
        tts = gTTS(text=text, lang='en', slow=False)
        speech = os.getcwd() + '/output.mp3'
        tts.save(speech)  # save a wav file
        return speech
    except:
        return ("data/check_internet.mp3")


def html_display(input_text):
    return f"""
            <h2><b style="color:#00008B"> {input_text}</b></h2>
            <br>
           """


def oled_display(input_text):
    oled.set_position(0, 0)
    oled.put_string("I heard you say: ")
    oled.set_position(3, 0)
    oled.put_string(f"{input_text}")


def reply(input_text):
    display(Audio(t2s(input_text), autoplay=True))
    sleep(3)


def temperature():
    return f'temperature is {temp_sensor.get_temperature():.2f} degree centigrade'


def lights(text_in):
    if 'on' in text_in and 'mood' not in text_in:
        if sensor_state['relay'] == 'off':
            relay.on()
            sensor_state.update(relay='on')
            return 'ok, turning the lights on'
        else:
            return 'lights are already on'
    elif 'off' in text_in and 'mood' not in text_in:
        if sensor_state['relay'] == 'on':
            relay.off()
            sensor_state.update(relay='off')
            return 'ok, turning the lights off'
    else:
        return 'lights are already off'


def mood(text_in):
    if 'happy' in text_in or 'Happy' in text_in:
        led_stick.clear()
        for i in range(10):
            led_stick.set_pixel(i, led_colors[i])
            led_stick.show()
        return 'I am glad you are having a good day. enjoy'
    elif 'calm' in text_in or 'Calm' in text_in:
        for i in range(10):
            led_stick.set_pixel(i, 0x00FFFF)
            led_stick.show()
        return 'being calm is good. hope it gets better',
    elif 'off' in text_in:
        led_stick.clear()
        return 'ok, turning the mood lights off',
    else:
        led_stick.clear()
        return 'please let me know if your in a happy or a calm mood'


def blinds(text_in):
    if 'open' in text_in:
        for x in range(10, 170, 10):
            servo.set_angular_position(x)
            sleep(0.1)
        return 'I opened the blinds for you'
    else:
        for x in range(160, 0, -10):
            servo.set_angular_position(x)
            sleep(0.1)
        return 'blinds are closed now'


def exit_application():
    clear_output()
    led_stick.clear()
    oled.clear_display()
    return 'ok, goodbye'


<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Notes before running the next cell </h4>
    <p>You can speak the commands when the 4 LEDs on the board are lit up. Please use commands that are short and clear ~5 seconds for best interaction</p>
    <b>Command list</b>
    <ul>
        <li> Light control</li>
        <ul><li><b>'Switch/turn on Lights'</b></li></ul>
        <ul><li><b>'Switch/turn off Lights'</b></li></ul>
        <li> Temperature sensing</li>
        <ul><li><b>What is the temperature?</b></li></ul>
        <li> Blinds control</li>
        <ul><li><b>Close the blinds please..</b></li></ul>
        <ul><li><b>Open the blinds please</b></li></ul>
        <li> Mood lighting</li>
        <ul><li><b>I am in a Happy mood / Set mood to happy</b></li></ul>
        <ul><li><b>I am in a Calm mood / Set mood to calm</b></li></ul>
        <li> Exit</li>
        <ul><li><b>Exit listening mode</b></li></ul>
    </ul>
</div>

In [8]:
count = 0
relay.off()
led_stick.clear()

while True:
    text_in = "-------------"
    oled.clear_display()
    hdisplay = display(HTML(''),
                       display_id=True)  # Initialize display variable
    [led.on() for led in base.leds]
    hdisplay.update(HTML(html_display("Listening...")))
    pAudio.record(5)
    pAudio.save("recording_1.wav")
    [led.off() for led in base.leds]
    hdisplay.update(HTML(html_display("Processing..")))

    harvard = sr.AudioFile('recording_1.wav')
    with harvard as source:
        audio = r.record(source)

    try:
        text_in = r.recognize_google(audio)
        count = 0
    except:
        count += 1
        if count < 50:
            clear_output()
            continue
        else:
            reply('turning off due to timeout')
            clear_output()
            oled.clear_display()
            relay.off()
            led_stick.clear()
            break

    oled_display(text_in)

    if re.search('weather|temperature', text_in):
        temp_response = temperature()
        reply(temp_response)
    elif re.search('lights|light', text_in) and not re.search('mood', text_in):
        light_response = lights(text_in)
        reply(light_response)
    elif re.search('mood|leds|RGB lights|RGB|happy|Happy|calm|Calm', text_in):
        mood_response = mood(text_in)
        reply(mood_response)
    elif re.search('blind|blinds|Blind|Blinds', text_in):
        blinds_response = blinds(text_in)
        reply(blinds_response)
    elif re.search('exit', text_in):
        exit_response = exit_application()
        reply(exit_response)
        clear_output()
        break
    else:
        reply("Sorry, I did not get that")

    sleep(4)
    clear_output()


Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----